# Quality control and filtering scATAC-seq data with Muon
**Author**: Adam Klie (last modified: 10/08/2023)<br>
***
**Description**: This script performs the initial quality control and filtering for a single cell ATAC-seq dataset using the Muon package. The output of this script is a filtered count matrix and a list of cells that passed QC. The filtered count matrix is used for downstream analysis. The list of cells that passed QC is used to filter the metadata file for downstream analysis.

In [27]:
# Imports
import os
import numpy as np
from scipy.io import mmread
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from muon import atac as ac

# CellCommander
from cellcommander.qc import atac

In [79]:
# Paths
sample = "MM129"
input_h5ad_path = f"/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/{sample}/outs/filtered_peak_bc_matrix.h5"
outdir_path = f"/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/annotation/26Oct23/cellcommander/{sample}/threshold_qc"
metadata_path = None

In [80]:
# If output directory does not exist, create it.
if not os.path.exists(outdir_path):
    os.makedirs(outdir_path)

In [81]:
# Load the dataset
adata = ac.read_10x_h5(input_h5ad_path)
adata.var_names_make_unique()
adata.uns["files"] = {}
adata.uns["files"]["fragments"] = f"/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_snATAC-seq/processed/23Oct23/cellranger/{sample}/outs/fragments.tsv.gz"
adata

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 20000 × 305581
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'files'

In [82]:
# Define run parameters
output_prefix = "threshold_qc"
total_counts_nmads = None
n_features_nmads = None
n_top_features = 20
pct_counts_in_top_features_nmads = None
total_counts_low = 1000
total_counts_hi = 60000
n_for_ns_calc = 1000
ns_nmads = None
ns_hi = 1.5
n_features_low = 1000
n_tss = 1000
tss_nmads = None
tss_low = 4
tss_hi = 18
min_cells_per_feature = 10
random_state = 1234

In [83]:
# I think Python 3.9 has problem with the regex currently in the atac_qc function
import pybiomart as pbm
dataset = pbm.Dataset(name="hsapiens_gene_ensembl", host="http://www.ensembl.org")
annot = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
filter = annot["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
annot = annot[~filter]
annot["Chromosome/scaffold name"] = "chr" + annot["Chromosome/scaffold name"]
annot.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
annot = annot[annot["Transcript_type"] == "protein_coding"]
annot = annot[
    annot.Chromosome.isin(
        ["chr" + str(i) for i in range(1, 23)] + ["chrX", "chrY", "chrM"]
    )
]
annot.head()

,Chromosome,Start,Strand,Gene,Transcript_type
234,chrY,22490397,1,PRY,protein_coding
259,chrY,12662368,1,USP9Y,protein_coding
261,chrY,12701231,1,USP9Y,protein_coding
264,chrY,12847045,1,USP9Y,protein_coding
288,chrY,22096007,-1,PRY2,protein_coding


In [84]:
# Perform the standard CellCommander QC
atac.atac_qc(
    adata=adata,
    n_top_features=n_top_features,
    pct_counts_in_top_features_nmads=pct_counts_in_top_features_nmads,
    n_for_ns_calc=n_for_ns_calc,
    ns_nmads=ns_nmads,
    ns_hi=ns_hi,
    n_tss=n_tss,
    tss_annot=annot,
    tss_nmads=tss_nmads,
    tss_hi=tss_hi,
    tss_low=tss_low,
    total_counts_nmads=total_counts_nmads,
    total_counts_low=total_counts_low,
    total_counts_hi=total_counts_hi,
    n_features_nmads=n_features_nmads,
    n_features_low=n_features_low,
    random_state=random_state,
)

Fetching Regions...: 100%|██████████| 1000/1000 [00:58<00:00, 17.14it/s]


In [85]:
# Make the standard CellCommander QC plots
atac.atac_qc_triplet_plot(
    adata=adata,
    outdir_path=outdir_path,
    output_prefix="pre",
    total_counts_bins=100,
    total_counts_low=total_counts_low,
    total_counts_hi=total_counts_hi,
    tss_low=tss_low,
    tss_hi=tss_hi,
)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:842: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  ax = sns.violinplot(
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:315: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(adata.obs[key]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_utils.py:714: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


In [86]:
# Filter the data
adata, filtered_bc = atac.atac_outlier_filter(
    adata,
    outlier_cols=["outlier", "mt_outlier"],
)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [87]:
# Save the barcodes
filtered_bc_path = os.path.join(outdir_path, "filtered_barcodes.txt")
filtered_bc.to_series().to_csv(filtered_bc_path, sep="\t", index=False, header=False)

In [88]:
# Make the standard CellCommander QC plots
atac.atac_qc_triplet_plot(
    adata=adata,
    outdir_path=outdir_path,
    output_prefix="post",
    total_counts_bins=100,
)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:842: FutureWarning: 

The `scale` parameter has been renamed and will be removed in v0.15.0. Pass `density_norm='width'` for the same effect.
  ax = sns.violinplot(
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:315: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(adata.obs[key]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_utils.py:714: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


In [89]:
# Optionally filter genes if min_cells is provided as arg
if min_cells_per_feature is not None:
    print(f"Filtering genes based on cell count. Number of genes before filtering: {adata.n_vars}")
    sc.pp.filter_genes(adata, min_cells=min_cells_per_feature)
    print(f"Number of genes after filtering: {adata.n_vars}")

Filtering genes based on cell count. Number of genes before filtering: 305581


/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Number of genes after filtering: 305579


In [90]:
# Write out h5ad
adata.write(os.path.join(outdir_path, f"{output_prefix}.h5ad"))

In [91]:
adata

AnnData object with n_obs × n_vars = 11962 × 305579
    obs: 'n_features_by_counts', 'total_counts', 'pct_counts_in_top_20_features', 'log_total_counts', 'nucleosome_signal', 'nuc_signal_filter', 'tss_score', 'outlier', 'atac_outlier'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'
    uns: 'files'

# DONE!

---